In [2]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 921.8/921.8 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [13]:
import os
import ultralytics
import numpy as np
from ultralytics import YOLO
import shutil
import cv2

In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
!cp "/content/drive/MyDrive/hong_kong/162-labeled-yolo.zip" "/content/"

In [6]:
!unzip "/content/162-labeled-yolo.zip"

Archive:  /content/162-labeled-yolo.zip
   creating: images/
   creating: labels/
  inflating: classes.txt             
  inflating: notes.json              
  inflating: images/01154da3-1976362.jpg  
  inflating: images/01c252f8-1574421.jpg  
  inflating: images/02c4415a-995801.jpg  
  inflating: images/02e731de-1106165.jpg  
  inflating: images/069f9c9b-118126.jpg  
  inflating: images/07e95446-2781056.jpg  
  inflating: images/080248d8-2031692.jpg  
  inflating: images/082737e5-2498460.jpg  
  inflating: images/0b45d160-3763432.jpg  
  inflating: images/0d1879f9-358748.jpg  
  inflating: images/11c97300-3515250.jpg  
  inflating: images/120ddac2-2674351.jpg  
  inflating: images/123adde7-3827966.jpg  
  inflating: images/165a1c35-538037.jpg  
  inflating: images/173ece9d-1098082.jpg  
  inflating: images/179a55c2-663285.jpg  
  inflating: images/18717882-2580309.jpg  
  inflating: images/1ac491f4-2569760.jpg  
  inflating: images/1ce34d36-3606882.jpg  
  inflating: images/1e2cd628-3

In [16]:
dataset_path = r"/content/"
processed_dataset_path = r"/content/processed_dataset3/"
images_path = os.path.join(processed_dataset_path, "images")
labels_path = os.path.join(processed_dataset_path, "labels")
os.makedirs(images_path, exist_ok=True)
os.makedirs(labels_path, exist_ok=True)

In [ ]:
def preprocess_image(image_path, target_size=(640, 640)):
    image = cv2.imread(image_path)
    if image is None:
        return None

    image = cv2.resize(image, target_size)

    image = cv2.GaussianBlur(image, (5, 5), 0)

    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    image = image / 255.0

    return (image * 255).astype(np.uint8)

image_files = os.listdir(os.path.join(dataset_path, "images"))
for image_file in image_files:
    image_path = os.path.join(dataset_path, "images", image_file)
    label_path = os.path.join(dataset_path, "labels", image_file.replace(".jpg", ".txt"))

    processed_image = preprocess_image(image_path)
    if processed_image is not None:
        cv2.imwrite(os.path.join(images_path, image_file), processed_image)

        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(labels_path, image_file.replace(".jpg", ".txt")))


In [8]:
import cv2
import numpy as np
import os
import shutil
import albumentations as A
from albumentations.pytorch import ToTensorV2

def preprocess_image(image_path, target_size=(640, 640)):
    image = cv2.imread(image_path)
    if image is None:
        return None

    # Resize the image
    image = cv2.resize(image, target_size)

    # Use Bilateral Filter instead of Gaussian Blur to keep edges
    image = cv2.bilateralFilter(image, 9, 75, 75)

    # Convert to LAB and apply CLAHE only if image is dark
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    if np.mean(l) < 100:  # Apply CLAHE only if the image is dark
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

    # Convert image to float32 and normalize (if using a deep learning framework)
    image = image.astype(np.float32) / 255.0

    return (image * 255).astype(np.uint8)

# Albumentations augmentation (optional)
augment = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
    A.GaussNoise(p=0.2),
    A.Rotate(limit=15, p=0.3),
])

image_files = os.listdir(os.path.join(dataset_path, "images"))
for image_file in image_files:
    image_path = os.path.join(dataset_path, "images", image_file)
    label_path = os.path.join(dataset_path, "labels", image_file.replace(".jpg", ".txt"))

    processed_image = preprocess_image(image_path)
    if processed_image is not None:
        augmented = augment(image=processed_image)["image"]
        cv2.imwrite(os.path.join(images_path, image_file), augmented)

        if os.path.exists(label_path):
            shutil.copy(label_path, os.path.join(labels_path, image_file.replace(".jpg", ".txt")))


In [17]:
import cv2
import numpy as np
import os
import shutil
import albumentations as A

# Define augmentation with polygon support
augment = A.Compose([
    A.HorizontalFlip(p=0.5),  # Flip
    A.RandomBrightnessContrast(p=0.2),  # Adjust brightness/contrast
    A.Rotate(limit=15, p=0.3),  # Rotate
], keypoint_params=A.KeypointParams(format="xy", remove_invisible=False))

def preprocess_and_augment(image_path, label_path, target_size=(640, 640)):
    image = cv2.imread(image_path)
    if image is None:
        return None, None

    # Read YOLO polygon label file
    polygons = []
    class_labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f.readlines():
                values = line.strip().split()
                if len(values) > 1:
                    class_id = int(values[0])
                    points = list(map(float, values[1:]))  # Polygon points
                    polygon = [(points[i], points[i+1]) for i in range(0, len(points), 2)]
                    polygons.append(polygon)
                    class_labels.append(class_id)

    # Apply augmentation
    augmented = augment(image=image, keypoints=[p for polygon in polygons for p in polygon])

    # Get augmented image and updated polygons
    aug_image = augmented["image"]
    aug_keypoints = augmented["keypoints"]

    # Reconstruct polygons
    new_polygons = []
    index = 0
    for polygon in polygons:
        new_polygons.append(aug_keypoints[index : index + len(polygon)])
        index += len(polygon)

    return aug_image, new_polygons, class_labels

# Process dataset
image_files = os.listdir(os.path.join(dataset_path, "images"))
for image_file in image_files:
    image_path = os.path.join(dataset_path, "images", image_file)
    label_path = os.path.join(dataset_path, "labels", image_file.replace(".jpg", ".txt"))

    # Preprocess and augment the image and labels
    processed_image, processed_polygons, processed_labels = preprocess_and_augment(image_path, label_path)

    if processed_image is not None:
        # Save augmented image
        cv2.imwrite(os.path.join(images_path, image_file), processed_image)

        # Save updated polygon label file
        if processed_polygons:
            label_save_path = os.path.join(labels_path, image_file.replace(".jpg", ".txt"))
            with open(label_save_path, "w") as f:
                for polygon, class_id in zip(processed_polygons, processed_labels):
                    flattened = [str(coord) for point in polygon for coord in point]
                    f.write(f"{class_id} " + " ".join(flattened) + "\n")


In [19]:
import cv2
import numpy as np
import os
import shutil
import albumentations as A

# Define augmentation with polygon support
augment = A.Compose([
    A.HorizontalFlip(p=0.5),  # Flip
    A.RandomBrightnessContrast(p=0.2),  # Adjust brightness/contrast
    A.Rotate(limit=15, p=0.3),  # Rotate
], keypoint_params=A.KeypointParams(format="xy", remove_invisible=False))

def preprocess_and_augment(image_path, label_path, target_size=(640, 640)):
    image = cv2.imread(image_path)
    if image is None:
        return None, None

    # Read YOLO polygon label file
    polygons = []
    class_labels = []
    if os.path.exists(label_path):
        with open(label_path, "r") as f:
            for line in f.readlines():
                values = line.strip().split()
                if len(values) > 1:
                    class_id = int(values[0])
                    points = list(map(float, values[1:]))  # Polygon points
                    polygon = [(points[i], points[i+1]) for i in range(0, len(points), 2)]
                    polygons.append(polygon)
                    class_labels.append(class_id)

    # Apply augmentation
    augmented = augment(image=image, keypoints=[p for polygon in polygons for p in polygon])

    # Get augmented image and updated polygons
    aug_image = augmented["image"]
    aug_keypoints = augmented["keypoints"]

    # Reconstruct polygons
    new_polygons = []
    index = 0
    for polygon in polygons:
        new_polygons.append(aug_keypoints[index : index + len(polygon)])
        index += len(polygon)

    return aug_image, new_polygons, class_labels

# Process dataset
image_files = os.listdir(os.path.join(dataset_path, "images"))
for image_file in image_files:
    image_path = os.path.join(dataset_path, "images", image_file)
    label_path = os.path.join(dataset_path, "labels", image_file.replace(".jpg", ".txt"))

    # Preprocess and augment the image and labels
    processed_image, processed_polygons, processed_labels = preprocess_and_augment(image_path, label_path)

    if processed_image is not None:
        # Save augmented image with a unique name (to avoid overwriting)
        aug_image_name = f"aug_{image_file}"
        cv2.imwrite(os.path.join(images_path, aug_image_name), processed_image)

        # Save updated polygon label file
        if processed_polygons:
            label_save_path = os.path.join(labels_path, aug_image_name.replace(".jpg", ".txt"))
            with open(label_save_path, "w") as f:
                for polygon, class_id in zip(processed_polygons, processed_labels):
                    flattened = [str(coord) for point in polygon for coord in point]
                    f.write(f"{class_id} " + " ".join(flattened) + "\n")


In [20]:
import os

folder_path = "/content/processed_dataset3/images"  # Change this to your folder path
image_count = len([f for f in os.listdir(folder_path) if f.endswith((".jpg", ".png", ".jpeg"))])

print(f"Number of images: {image_count}")


Number of images: 324


In [21]:
yaml_path = os.path.join(processed_dataset_path, "data.yaml")
classes_file = os.path.join(dataset_path, "classes.txt")

if os.path.exists(classes_file):
    with open(classes_file, "r") as f:
        class_names = [line.strip() for line in f.readlines()]
else:
    class_names = ["class_0", "class_1", "class_2", "class_3"]

yaml_content = f"""
path: {processed_dataset_path}
train: images
val: images
test: images

nc: {len(class_names)}
names: {class_names}
"""

with open(yaml_path, "w") as f:
    f.write(yaml_content.strip())


In [22]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.backends.cudnn.version())
print(torch.cuda.is_available())
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print(torch.version.cuda)
print(ultralytics.__version__)
model = YOLO("yolo11n-seg.pt").to("cuda")
torch.compile(model)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True


2.5.1+cu124
12.4
90100
True
True
Tesla T4
12.4
8.3.79


In [23]:
if __name__ == '__main__':
    model.train(
        data=yaml_path,
        epochs=60,
        imgsz=640,
        batch=16,
        device="0",
        optimizer="AdamW",
        lr0=0.01,
        lrf=0.001,
        momentum=0.937,
        weight_decay=0.0005,
        cos_lr=True,
        patience=10
)

Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=/content/processed_dataset3/data.yaml, epochs=60, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

train: Scanning /content/processed_dataset3/labels... 324 images, 0 backgrounds, 218 corrupt: 100%|██████████| 324/324 [00:00<00:00, 1876.53it/s]

train: WARNING ⚠️ /content/processed_dataset3/images/01c252f8-1574421.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     382.36      382.71      382.74      382.65      382.38      382.36      382.11      382.91      382.68]
train: WARNING ⚠️ /content/processed_dataset3/images/02c4415a-995801.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     19.226]
train: WARNING ⚠️ /content/processed_dataset3/images/069f9c9b-118126.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     382.52]
train: WARNING ⚠️ /content/processed_dataset3/images/07e95446-2781056.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     510.57]
train: WARNING ⚠️ /content/processed_dataset3/images/080248d8-2031692.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     496.58      1.0339]
train: WARNING ⚠️ /content/processed_dataset3/images/082737e5-2498460.jpg: igno


val: Scanning /content/processed_dataset3/labels.cache... 324 images, 0 backgrounds, 218 corrupt: 100%|██████████| 324/324 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/processed_dataset3/images/01c252f8-1574421.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     382.36      382.71      382.74      382.65      382.38      382.36      382.11      382.91      382.68]
train: WARNING ⚠️ /content/processed_dataset3/images/02c4415a-995801.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     19.226]
train: WARNING ⚠️ /content/processed_dataset3/images/069f9c9b-118126.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     382.52]
train: WARNING ⚠️ /content/processed_dataset3/images/07e95446-2781056.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     510.57]
train: WARNING ⚠️ /content/processed_dataset3/images/080248d8-2031692.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     496.58      1.0339]
train: WARNING ⚠️ /content/processed_dataset3/images/082737e5-2498460.jpg: igno

KeyboardInterrupt: 

In [24]:
if __name__ == '__main__':
    model.train(
        data=yaml_path,
        epochs=100,  # More epochs
        imgsz=1024,  # Larger image size
        batch=8,  # Smaller batch size
        device="0",
        optimizer="SGD",  # Try SGD if AdamW doesn't work
        lr0=0.002,  # Lower learning rate
        lrf=0.0005,  # Fine-tune LR final value
        momentum=0.9,  # Slightly lower momentum
        weight_decay=0.0005,
        cos_lr=True,
        patience=20  # More patience for early stopping
    )


Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=segment, mode=train, model=yolo11n-seg.pt, data=/content/processed_dataset3/data.yaml, epochs=100, time=None, patience=20, batch=8, imgsz=1024, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

train: Scanning /content/processed_dataset3/labels.cache... 324 images, 0 backgrounds, 218 corrupt: 100%|██████████| 324/324 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/processed_dataset3/images/01c252f8-1574421.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     382.36      382.71      382.74      382.65      382.38      382.36      382.11      382.91      382.68]
train: WARNING ⚠️ /content/processed_dataset3/images/02c4415a-995801.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     19.226]
train: WARNING ⚠️ /content/processed_dataset3/images/069f9c9b-118126.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     382.52]
train: WARNING ⚠️ /content/processed_dataset3/images/07e95446-2781056.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     510.57]
train: WARNING ⚠️ /content/processed_dataset3/images/080248d8-2031692.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     496.58      1.0339]
train: WARNING ⚠️ /content/processed_dataset3/images/082737e5-2498460.jpg: igno


val: Scanning /content/processed_dataset3/labels.cache... 324 images, 0 backgrounds, 218 corrupt: 100%|██████████| 324/324 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/processed_dataset3/images/01c252f8-1574421.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     382.36      382.71      382.74      382.65      382.38      382.36      382.11      382.91      382.68]
train: WARNING ⚠️ /content/processed_dataset3/images/02c4415a-995801.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     19.226]
train: WARNING ⚠️ /content/processed_dataset3/images/069f9c9b-118126.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     382.52]
train: WARNING ⚠️ /content/processed_dataset3/images/07e95446-2781056.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     510.57]
train: WARNING ⚠️ /content/processed_dataset3/images/080248d8-2031692.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     496.58      1.0339]
train: WARNING ⚠️ /content/processed_dataset3/images/082737e5-2498460.jpg: igno

...


      Epoch    GPU_mem   box_loss   seg_loss   cls_loss   dfl_loss  Instances       Size


    100/100      3.49G      0.911      1.299      3.662       1.41          2       1024: 100%|██████████| 14/14 [00:05<00:00,  2.50it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.65it/s]


                   all        106        132       0.78      0.879      0.909      0.677       0.78      0.879      0.911      0.673

100 epochs completed in 0.250 hours.
Optimizer stripped from runs/segment/train3/weights/last.pt, 6.1MB
Optimizer stripped from runs/segment/train3/weights/best.pt, 6.1MB

Validating runs/segment/train3/weights/best.pt...
Ultralytics 8.3.79 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n-seg summary (fused): 113 layers, 2,835,348 parameters, 0 gradients, 10.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:07<00:00,  1.03s/it]


                   all        106        132        0.9      0.941      0.972      0.742        0.9      0.941      0.972      0.737
            fried_rice         28         28      0.835      0.964      0.966      0.733      0.835      0.964      0.966      0.788
             ice_cream         27         38      0.833      0.842      0.939      0.696      0.833      0.842      0.939      0.643
                 pizza         19         19      0.967          1      0.995       0.73      0.967          1      0.995      0.785
                 sushi         32         47      0.963      0.957      0.988      0.808      0.963      0.957      0.988      0.734
Speed: 0.6ms preprocess, 10.8ms inference, 0.0ms loss, 19.4ms postprocess per image
Results saved to runs/segment/train3


In [ ]:
image_path = r"/content/images"
model = YOLO(r"/content/runs/segment/train/weights/best.pt")
results = model.predict(r"/content/images/01154da3-1976362.jpg")



image 1/1 /content/images/01154da3-1976362.jpg: 480x640 (no detections), 29.2ms
Speed: 11.6ms preprocess, 29.2ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)
